In [1]:
def get_memcache_client(memcache_host="memcache", memcache_port=11211):
    from pymemcache import serde
    from pymemcache.client import Client as MemcacheClient

    memcache_host = "memcache"
    memcache_port = 11211

    client = MemcacheClient(
        (memcache_host, memcache_port),
        serializer=serde.python_memcache_serializer,
        deserializer=serde.python_memcache_deserializer,
    )
    return client


def list_memcache_keys(client=None):
    from six import ensure_str

    # List all keys in memcache
    if not client:
        client = get_memcache_client()
    keys = {}
    for key, val in client.stats("items").items():
        _, slab, field = ensure_str(key).split(":")
        if field != "number" or val == 0:
            continue
        item_request = client.stats("cachedump", slab, str(val + 10))
        for record, details in item_request.items():
            keys[ensure_str(record)] = ensure_str(details)
    return keys


def delete_memcache_key(key_prefix: str, client=None):
    """Delete all keys in memcache that start with the prefix

    If running in Jupyter, the prefix is your username.
    If running in Airflow, the prefix is the DAG ID.
    """
    from pymemcache import serde
    from pymemcache.client import Client as MemcacheClient

    if not client:
        client = get_memcache_client()

    keys = list_memcache_keys(client)

    print(f"Found {len(keys)} keys in memcache")
    print(f"Deleting keys that start with: {key_prefix}")

    # Delete all keys that start with the prefix
    for key in keys:
        if key.startswith(key_prefix):
            client.delete(key)
            print(f"Deleted key: {key}")
    pass

In [2]:
# First see which keys are in memcache
client = get_memcache_client()
keys = list_memcache_keys(client)
print(f"Found {len(keys)} keys in memcache")
print(keys)

Found 0 keys in memcache
{}


In [ ]:
# Now delete all keys that start with the prefix
prefix = "xxxxxxxx"
delete_memcache_key(prefix, client)

: 